In [29]:
import pandas as pd
import pickle

invoices_path = "/Users/melineafrikyan/Desktop/data/invoices_new.pkl"
expired_invoices_path = "/Users/melineafrikyan/Desktop/data/expired_invoices.txt"

class DataExtractor:
    def __init__(self, invoices_path, expired_invoices_path):
        self.invoices_path = invoices_path
        self.expired_invoices_path = expired_invoices_path
    
    def load_expired_invoices(self):
        with open(self.expired_invoices_path, 'r') as file:
            expired_ids = file.read().strip().split(", ")
        return set(map(str, expired_ids))
    
    def load_invoices(self):
        with open(self.invoices_path, 'rb') as file:
            invoices = pickle.load(file)
        return invoices
    
    def transform_data(self):
        expired_invoices = self.load_expired_invoices()
        invoices = self.load_invoices()

        flat_data = []
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        
        for invoice in invoices:
            invoice_id = str(invoice.get('id', 0)) 
            created_on = pd.to_datetime(invoice.get('created_on', '1900-01-01'), errors='coerce')
            invoice_name = invoice.get('name', 'NA') 

            
            def safe_numeric(value):
                try:
                    return float(value)
                except (ValueError, TypeError):
                    return 0

           
            invoice_total = sum(
                safe_numeric(item['item'].get('unit_price', 0)) * safe_numeric(item.get('quantity', 0))
                for item in invoice.get('items', [])
            )
            is_expired = invoice_id in expired_invoices
            
           
            for item in invoice.get('items', []):
                item_details = item.get('item', {})
                invoiceitem_id = item_details.get('id', 0)
                item_name = item_details.get('name', 'NA') 
                type_string = type_conversion.get(item_details.get('type', 3), 'Other')

                unit_price = safe_numeric(item_details.get('unit_price', 0))
                quantity = safe_numeric(item.get('quantity', 0))
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total if invoice_total else 0

                flat_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoice_name': item_name,  
                    'type': type_string,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })

        df = pd.DataFrame(flat_data)
        return df.sort_values(by=['invoice_id', 'invoiceitem_id'])

data_extractor = DataExtractor(invoices_path, expired_invoices_path)
dataframe = data_extractor.transform_data()
print(dataframe)





    invoice_id created_on  invoiceitem_id invoice_name       type  unit_price  total_price  percentage_in_invoice  is_expired
292     301695 2019-04-26          103215    ii_103215      Other       135.0        945.0               0.374851       False
290     301695 2019-04-26          166227    ii_166227  Equipment       118.0        708.0               0.280841       False
293     301695 2019-04-26          171394    ii_171394      Other       128.0        128.0               0.050774       False
291     301695 2019-04-26          195625    ii_195625      Other       148.0        740.0               0.293534       False
130     304245 2019-03-17          121446    ii_121446    Service       158.0        632.0               0.128429       False
135     304245 2019-03-17          135235    ii_135235    Service       151.0        906.0               0.184109       False
132     304245 2019-03-17          143674    ii_143674    Service       176.0        528.0               0.107295     